In [1]:
!ls

file_1_mapping_new.csv          node-label.csv
file_2_edge_new.csv             nodeidx2uniquekey.csv
file_3_feature.csv              output.json
input_files_preprocessing.ipynb


In [2]:
import pandas as pd
import numpy as np

## Create key 'node_ids'

In [3]:
df_file_1_mapping=pd.read_csv('file_1_mapping_new.csv')
df_file_1_mapping

,fac_no,code_no,fac_type,fac_name
0,J59A7436,R-0201,A1,力成科技股份有限公司3C廠
1,J59A0579,R-0201,A1,力成科技股份有限公司大同廠
2,J59B1009,R-0201,A1,力成科技股份有限公司文化一廠
3,O17K9082,R-0201,A1,力成科技股份有限公司竹科分公司
4,J5909547,R-0201,A1,力成科技股份有限公司湖口廠
...,...,...,...,...
541,89523396,220099:其他未列名塑膠製品,S,鴻升包裝器材事業有限公司
542,59132956,183799:其他塑膠粒,S,豐隆塑膠加工廠
543,NaN,183799:其他塑膠粒,S,TIANJIN JIUFU(天津九福國際物流有限公司)
544,NaN,183799:其他塑膠粒,S,Xiamen Luhai (廈門陸海環保股份有限公司)


In [4]:
df_file_1_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fac_no    541 non-null    object
 1   code_no   546 non-null    object
 2   fac_type  546 non-null    object
 3   fac_name  546 non-null    object
dtypes: object(4)
memory usage: 17.2+ KB


In [5]:
# replace NaN values with an empty string
df_file_1_mapping=df_file_1_mapping.fillna('')
df_file_1_mapping

,fac_no,code_no,fac_type,fac_name
0,J59A7436,R-0201,A1,力成科技股份有限公司3C廠
1,J59A0579,R-0201,A1,力成科技股份有限公司大同廠
2,J59B1009,R-0201,A1,力成科技股份有限公司文化一廠
3,O17K9082,R-0201,A1,力成科技股份有限公司竹科分公司
4,J5909547,R-0201,A1,力成科技股份有限公司湖口廠
...,...,...,...,...
541,89523396,220099:其他未列名塑膠製品,S,鴻升包裝器材事業有限公司
542,59132956,183799:其他塑膠粒,S,豐隆塑膠加工廠
543,,183799:其他塑膠粒,S,TIANJIN JIUFU(天津九福國際物流有限公司)
544,,183799:其他塑膠粒,S,Xiamen Luhai (廈門陸海環保股份有限公司)


In [6]:
df_file_1_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fac_no    546 non-null    object
 1   code_no   546 non-null    object
 2   fac_type  546 non-null    object
 3   fac_name  546 non-null    object
dtypes: object(4)
memory usage: 17.2+ KB


In [7]:
# use agg() to concat 'fac_no', 'code_no', 'fac_type', 'fac_name' columns
df_file_1_mapping['unique_key']=df_file_1_mapping[['fac_no', 'code_no', 'fac_type', 'fac_name']].agg('_'.join, axis=1)
df_file_1_mapping

,fac_no,code_no,fac_type,fac_name,unique_key
0,J59A7436,R-0201,A1,力成科技股份有限公司3C廠,J59A7436_R-0201_A1_力成科技股份有限公司3C廠
1,J59A0579,R-0201,A1,力成科技股份有限公司大同廠,J59A0579_R-0201_A1_力成科技股份有限公司大同廠
2,J59B1009,R-0201,A1,力成科技股份有限公司文化一廠,J59B1009_R-0201_A1_力成科技股份有限公司文化一廠
3,O17K9082,R-0201,A1,力成科技股份有限公司竹科分公司,O17K9082_R-0201_A1_力成科技股份有限公司竹科分公司
4,J5909547,R-0201,A1,力成科技股份有限公司湖口廠,J5909547_R-0201_A1_力成科技股份有限公司湖口廠
...,...,...,...,...,...
541,89523396,220099:其他未列名塑膠製品,S,鴻升包裝器材事業有限公司,89523396_220099:其他未列名塑膠製品_S_鴻升包裝器材事業有限公司
542,59132956,183799:其他塑膠粒,S,豐隆塑膠加工廠,59132956_183799:其他塑膠粒_S_豐隆塑膠加工廠
543,,183799:其他塑膠粒,S,TIANJIN JIUFU(天津九福國際物流有限公司),_183799:其他塑膠粒_S_TIANJIN JIUFU(天津九福國際物流有限公司)
544,,183799:其他塑膠粒,S,Xiamen Luhai (廈門陸海環保股份有限公司),_183799:其他塑膠粒_S_Xiamen Luhai (廈門陸海環保股份有限公司)


In [8]:
df_file_1_mapping=df_file_1_mapping.drop(['fac_no', 'code_no', 'fac_type', 'fac_name'], axis=1)
df_file_1_mapping

,unique_key
0,J59A7436_R-0201_A1_力成科技股份有限公司3C廠
1,J59A0579_R-0201_A1_力成科技股份有限公司大同廠
2,J59B1009_R-0201_A1_力成科技股份有限公司文化一廠
3,O17K9082_R-0201_A1_力成科技股份有限公司竹科分公司
4,J5909547_R-0201_A1_力成科技股份有限公司湖口廠
...,...
541,89523396_220099:其他未列名塑膠製品_S_鴻升包裝器材事業有限公司
542,59132956_183799:其他塑膠粒_S_豐隆塑膠加工廠
543,_183799:其他塑膠粒_S_TIANJIN JIUFU(天津九福國際物流有限公司)
544,_183799:其他塑膠粒_S_Xiamen Luhai (廈門陸海環保股份有限公司)


In [9]:
# export to csv
# index_label: column label for index column(s) if desired
df_file_1_mapping.to_csv('nodeidx2uniquekey.csv', index_label='node_idx')

In [10]:
node_ids={}

for i in range(546):
    key=df_file_1_mapping.iloc[[i], [0]].values
    node_ids[key[0][0]]=i

In [11]:
node_ids

{'J59A7436_R-0201_A1_力成科技股份有限公司3C廠': 0,
 'J59A0579_R-0201_A1_力成科技股份有限公司大同廠': 1,
 'J59B1009_R-0201_A1_力成科技股份有限公司文化一廠': 2,
 'O17K9082_R-0201_A1_力成科技股份有限公司竹科分公司': 3,
 'J5909547_R-0201_A1_力成科技股份有限公司湖口廠': 4,
 'J5700946_R-0201_A1_力成科技股份有限公司新埔一廠': 5,
 'O17M3925_R-0201_A1_力晶積成電子製造股份有限公司P1/2': 6,
 'L8601927_R-0201_A1_三元紙業股份有限公司': 7,
 'F13A4902_R-0201_A1_三龍產業股份有限公司': 8,
 'B9100601_R-0201_A1_三櫻企業股份有限公司': 9,
 'B2304790_R-0201_A1_大立光電股份有限公司': 10,
 'B24B8998_R-0201_A1_大立光電股份有限公司七之一廠': 11,
 'B24B8988_R-0201_A1_大立光電股份有限公司七廠': 12,
 'B2408711_R-0201_A1_大立光電股份有限公司二廠': 13,
 'B24B4481_R-0201_A1_大立光電股份有限公司六廠': 14,
 'B24A7498_R-0201_A1_大立光電股份有限公司精密機械園區廠': 15,
 'H4508927_R-0201_A1_大陸營建廢棄物共同清除處理股份有限公司': 16,
 'H4406277_R-0201_A1_大輿股份有限公司': 17,
 'N0910827_R-0201_A1_大豐環保科技股份有限公司': 18,
 'J55C4763_R-0201_A1_大豐環保科技股份有限公司新竹分公司': 19,
 'F0800317_R-0201_A1_中國砂輪企業股份有限公司鶯歌廠': 20,
 'H4500725_R-0201_A1_中華汽車工業股份有限公司楊梅廠': 21,
 'H53B8032_R-0201_A1_中華紙漿股份有限公司觀音廠': 22,
 'H53B2064_R-0201_A1_中鼎化工股份有限公司桃科廠': 23,
 'H4895229_R-0201_A

## Create keys '0' to '167'

### Create key 'edges'

In [12]:
df_file_2_edge=pd.read_csv('file_2_edge_new.csv')
df_file_2_edge

,fac_no1,code_no1,fac_type1,fac_name1,fac_no2,code_no2,fac_type2,fac_name2,9901,9902,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,H45B6696,R-0201,RM,永茂環科股份有限公司,H45B6696,000099:其他,RP,永茂環科股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H47A9742,R-0201,RM,柯富伊企業有限公司(二廠),H47A9742,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司(二廠),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H51A9221,R-0201,RM,加暘環保科技有限公司,H51A9221,220099:其他未列名塑膠製品,RP,加暘環保科技有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K8200222,R-0201,RM,連泰紙業股份有限公司,K8200222,183799:其他塑膠粒,RP,連泰紙業股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H47A6728,R-0201,RM,柯富伊企業有限公司,H47A6728,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,A3995069,R-0201,C,達和廢棄物清除股份有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,A45A1094,R-0201,C,昌久金屬科技有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,B0200217,R-0201,C,佳霖環保工程有限公司,K8200222,R-0201,RM,連泰紙業股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,B0413983,R-0201,C,世和環保有限公司,H45B6696,R-0201,RM,永茂環科股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_file_2_edge=df_file_2_edge[['fac_no1', 'code_no1', 'fac_type1', 'fac_name1', 'fac_no2', 'code_no2', 'fac_type2', 'fac_name2']]
df_file_2_edge

,fac_no1,code_no1,fac_type1,fac_name1,fac_no2,code_no2,fac_type2,fac_name2
0,H45B6696,R-0201,RM,永茂環科股份有限公司,H45B6696,000099:其他,RP,永茂環科股份有限公司
1,H47A9742,R-0201,RM,柯富伊企業有限公司(二廠),H47A9742,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司(二廠)
2,H51A9221,R-0201,RM,加暘環保科技有限公司,H51A9221,220099:其他未列名塑膠製品,RP,加暘環保科技有限公司
3,K8200222,R-0201,RM,連泰紙業股份有限公司,K8200222,183799:其他塑膠粒,RP,連泰紙業股份有限公司
4,H47A6728,R-0201,RM,柯富伊企業有限公司,H47A6728,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司
...,...,...,...,...,...,...,...,...
643,A3995069,R-0201,C,達和廢棄物清除股份有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠
644,A45A1094,R-0201,C,昌久金屬科技有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠
645,B0200217,R-0201,C,佳霖環保工程有限公司,K8200222,R-0201,RM,連泰紙業股份有限公司
646,B0413983,R-0201,C,世和環保有限公司,H45B6696,R-0201,RM,永茂環科股份有限公司


In [14]:
df_file_2_edge.info() # 1 null value in 'fac_no1', 3 null values in 'fac_no2'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fac_no1    647 non-null    object
 1   code_no1   648 non-null    object
 2   fac_type1  648 non-null    object
 3   fac_name1  648 non-null    object
 4   fac_no2    645 non-null    object
 5   code_no2   648 non-null    object
 6   fac_type2  648 non-null    object
 7   fac_name2  648 non-null    object
dtypes: object(8)
memory usage: 40.6+ KB


In [15]:
# replace NaN values with an empty string
df_file_2_edge=df_file_2_edge.fillna('')
df_file_2_edge

,fac_no1,code_no1,fac_type1,fac_name1,fac_no2,code_no2,fac_type2,fac_name2
0,H45B6696,R-0201,RM,永茂環科股份有限公司,H45B6696,000099:其他,RP,永茂環科股份有限公司
1,H47A9742,R-0201,RM,柯富伊企業有限公司(二廠),H47A9742,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司(二廠)
2,H51A9221,R-0201,RM,加暘環保科技有限公司,H51A9221,220099:其他未列名塑膠製品,RP,加暘環保科技有限公司
3,K8200222,R-0201,RM,連泰紙業股份有限公司,K8200222,183799:其他塑膠粒,RP,連泰紙業股份有限公司
4,H47A6728,R-0201,RM,柯富伊企業有限公司,H47A6728,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司
...,...,...,...,...,...,...,...,...
643,A3995069,R-0201,C,達和廢棄物清除股份有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠
644,A45A1094,R-0201,C,昌久金屬科技有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠
645,B0200217,R-0201,C,佳霖環保工程有限公司,K8200222,R-0201,RM,連泰紙業股份有限公司
646,B0413983,R-0201,C,世和環保有限公司,H45B6696,R-0201,RM,永茂環科股份有限公司


In [16]:
df_file_2_edge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fac_no1    648 non-null    object
 1   code_no1   648 non-null    object
 2   fac_type1  648 non-null    object
 3   fac_name1  648 non-null    object
 4   fac_no2    648 non-null    object
 5   code_no2   648 non-null    object
 6   fac_type2  648 non-null    object
 7   fac_name2  648 non-null    object
dtypes: object(8)
memory usage: 40.6+ KB


In [17]:
# use agg() to concat 'fac_no1', 'code_no1', 'fac_type1', 'fac_name1' columns
df_file_2_edge['unique_key1']=df_file_2_edge[['fac_no1', 'code_no1', 'fac_type1', 'fac_name1']].agg('_'.join, axis=1)
df_file_2_edge

,fac_no1,code_no1,fac_type1,fac_name1,fac_no2,code_no2,fac_type2,fac_name2,unique_key1
0,H45B6696,R-0201,RM,永茂環科股份有限公司,H45B6696,000099:其他,RP,永茂環科股份有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司
1,H47A9742,R-0201,RM,柯富伊企業有限公司(二廠),H47A9742,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司(二廠),H47A9742_R-0201_RM_柯富伊企業有限公司(二廠)
2,H51A9221,R-0201,RM,加暘環保科技有限公司,H51A9221,220099:其他未列名塑膠製品,RP,加暘環保科技有限公司,H51A9221_R-0201_RM_加暘環保科技有限公司
3,K8200222,R-0201,RM,連泰紙業股份有限公司,K8200222,183799:其他塑膠粒,RP,連泰紙業股份有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司
4,H47A6728,R-0201,RM,柯富伊企業有限公司,H47A6728,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司,H47A6728_R-0201_RM_柯富伊企業有限公司
...,...,...,...,...,...,...,...,...,...
643,A3995069,R-0201,C,達和廢棄物清除股份有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,A3995069_R-0201_C_達和廢棄物清除股份有限公司
644,A45A1094,R-0201,C,昌久金屬科技有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,A45A1094_R-0201_C_昌久金屬科技有限公司
645,B0200217,R-0201,C,佳霖環保工程有限公司,K8200222,R-0201,RM,連泰紙業股份有限公司,B0200217_R-0201_C_佳霖環保工程有限公司
646,B0413983,R-0201,C,世和環保有限公司,H45B6696,R-0201,RM,永茂環科股份有限公司,B0413983_R-0201_C_世和環保有限公司


In [18]:
# use agg() to concat 'fac_no2', 'code_no2', 'fac_type2', 'fac_name2' columns
df_file_2_edge['unique_key2']=df_file_2_edge[['fac_no2', 'code_no2', 'fac_type2', 'fac_name2']].agg('_'.join, axis=1)
df_file_2_edge

,fac_no1,code_no1,fac_type1,fac_name1,fac_no2,code_no2,fac_type2,fac_name2,unique_key1,unique_key2
0,H45B6696,R-0201,RM,永茂環科股份有限公司,H45B6696,000099:其他,RP,永茂環科股份有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司,H45B6696_000099:其他_RP_永茂環科股份有限公司
1,H47A9742,R-0201,RM,柯富伊企業有限公司(二廠),H47A9742,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司(二廠),H47A9742_R-0201_RM_柯富伊企業有限公司(二廠),H47A9742_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司(二廠)
2,H51A9221,R-0201,RM,加暘環保科技有限公司,H51A9221,220099:其他未列名塑膠製品,RP,加暘環保科技有限公司,H51A9221_R-0201_RM_加暘環保科技有限公司,H51A9221_220099:其他未列名塑膠製品_RP_加暘環保科技有限公司
3,K8200222,R-0201,RM,連泰紙業股份有限公司,K8200222,183799:其他塑膠粒,RP,連泰紙業股份有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司,K8200222_183799:其他塑膠粒_RP_連泰紙業股份有限公司
4,H47A6728,R-0201,RM,柯富伊企業有限公司,H47A6728,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司,H47A6728_R-0201_RM_柯富伊企業有限公司,H47A6728_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司
...,...,...,...,...,...,...,...,...,...,...
643,A3995069,R-0201,C,達和廢棄物清除股份有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,A3995069_R-0201_C_達和廢棄物清除股份有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠
644,A45A1094,R-0201,C,昌久金屬科技有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,A45A1094_R-0201_C_昌久金屬科技有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠
645,B0200217,R-0201,C,佳霖環保工程有限公司,K8200222,R-0201,RM,連泰紙業股份有限公司,B0200217_R-0201_C_佳霖環保工程有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司
646,B0413983,R-0201,C,世和環保有限公司,H45B6696,R-0201,RM,永茂環科股份有限公司,B0413983_R-0201_C_世和環保有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司


In [19]:
df_file_2_edge=df_file_2_edge[['unique_key1', 'unique_key2']]
df_file_2_edge

,unique_key1,unique_key2
0,H45B6696_R-0201_RM_永茂環科股份有限公司,H45B6696_000099:其他_RP_永茂環科股份有限公司
1,H47A9742_R-0201_RM_柯富伊企業有限公司(二廠),H47A9742_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司(二廠)
2,H51A9221_R-0201_RM_加暘環保科技有限公司,H51A9221_220099:其他未列名塑膠製品_RP_加暘環保科技有限公司
3,K8200222_R-0201_RM_連泰紙業股份有限公司,K8200222_183799:其他塑膠粒_RP_連泰紙業股份有限公司
4,H47A6728_R-0201_RM_柯富伊企業有限公司,H47A6728_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司
...,...,...
643,A3995069_R-0201_C_達和廢棄物清除股份有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠
644,A45A1094_R-0201_C_昌久金屬科技有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠
645,B0200217_R-0201_C_佳霖環保工程有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司
646,B0413983_R-0201_C_世和環保有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司


In [20]:
!ls

file_1_mapping_new.csv          node-label.csv
file_2_edge_new.csv             nodeidx2uniquekey.csv
file_3_feature.csv              output.json
input_files_preprocessing.ipynb


In [21]:
df_idx=pd.read_csv('nodeidx2uniquekey.csv')
df_idx

,node_idx,unique_key
0,0,J59A7436_R-0201_A1_力成科技股份有限公司3C廠
1,1,J59A0579_R-0201_A1_力成科技股份有限公司大同廠
2,2,J59B1009_R-0201_A1_力成科技股份有限公司文化一廠
3,3,O17K9082_R-0201_A1_力成科技股份有限公司竹科分公司
4,4,J5909547_R-0201_A1_力成科技股份有限公司湖口廠
...,...,...
541,541,89523396_220099:其他未列名塑膠製品_S_鴻升包裝器材事業有限公司
542,542,59132956_183799:其他塑膠粒_S_豐隆塑膠加工廠
543,543,_183799:其他塑膠粒_S_TIANJIN JIUFU(天津九福國際物流有限公司)
544,544,_183799:其他塑膠粒_S_Xiamen Luhai (廈門陸海環保股份有限公司)


In [22]:
# find node_idx1 for unique_key1
df_file_2_edge=pd.merge(df_file_2_edge, df_idx, how='left', left_on='unique_key1', right_on='unique_key')
df_file_2_edge

,unique_key1,unique_key2,node_idx,unique_key
0,H45B6696_R-0201_RM_永茂環科股份有限公司,H45B6696_000099:其他_RP_永茂環科股份有限公司,477,H45B6696_R-0201_RM_永茂環科股份有限公司
1,H47A9742_R-0201_RM_柯富伊企業有限公司(二廠),H47A9742_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司(二廠),482,H47A9742_R-0201_RM_柯富伊企業有限公司(二廠)
2,H51A9221_R-0201_RM_加暘環保科技有限公司,H51A9221_220099:其他未列名塑膠製品_RP_加暘環保科技有限公司,475,H51A9221_R-0201_RM_加暘環保科技有限公司
3,K8200222_R-0201_RM_連泰紙業股份有限公司,K8200222_183799:其他塑膠粒_RP_連泰紙業股份有限公司,486,K8200222_R-0201_RM_連泰紙業股份有限公司
4,H47A6728_R-0201_RM_柯富伊企業有限公司,H47A6728_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司,481,H47A6728_R-0201_RM_柯富伊企業有限公司
...,...,...,...,...
643,A3995069_R-0201_C_達和廢棄物清除股份有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠,433,A3995069_R-0201_C_達和廢棄物清除股份有限公司
644,A45A1094_R-0201_C_昌久金屬科技有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠,364,A45A1094_R-0201_C_昌久金屬科技有限公司
645,B0200217_R-0201_C_佳霖環保工程有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司,356,B0200217_R-0201_C_佳霖環保工程有限公司
646,B0413983_R-0201_C_世和環保有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司,325,B0413983_R-0201_C_世和環保有限公司


In [23]:
df_file_2_edge=df_file_2_edge[['unique_key1', 'unique_key2', 'node_idx']]
df_file_2_edge

,unique_key1,unique_key2,node_idx
0,H45B6696_R-0201_RM_永茂環科股份有限公司,H45B6696_000099:其他_RP_永茂環科股份有限公司,477
1,H47A9742_R-0201_RM_柯富伊企業有限公司(二廠),H47A9742_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司(二廠),482
2,H51A9221_R-0201_RM_加暘環保科技有限公司,H51A9221_220099:其他未列名塑膠製品_RP_加暘環保科技有限公司,475
3,K8200222_R-0201_RM_連泰紙業股份有限公司,K8200222_183799:其他塑膠粒_RP_連泰紙業股份有限公司,486
4,H47A6728_R-0201_RM_柯富伊企業有限公司,H47A6728_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司,481
...,...,...,...
643,A3995069_R-0201_C_達和廢棄物清除股份有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠,433
644,A45A1094_R-0201_C_昌久金屬科技有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠,364
645,B0200217_R-0201_C_佳霖環保工程有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司,356
646,B0413983_R-0201_C_世和環保有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司,325


In [24]:
df_file_2_edge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 647
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   unique_key1  648 non-null    object
 1   unique_key2  648 non-null    object
 2   node_idx     648 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 20.2+ KB


In [25]:
# find node_idx2 for unique_key2
df_file_2_edge=pd.merge(df_file_2_edge, df_idx, how='left', left_on='unique_key2', right_on='unique_key')
df_file_2_edge

,unique_key1,unique_key2,node_idx_x,node_idx_y,unique_key
0,H45B6696_R-0201_RM_永茂環科股份有限公司,H45B6696_000099:其他_RP_永茂環科股份有限公司,477,494,H45B6696_000099:其他_RP_永茂環科股份有限公司
1,H47A9742_R-0201_RM_柯富伊企業有限公司(二廠),H47A9742_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司(二廠),482,496,H47A9742_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司(二廠)
2,H51A9221_R-0201_RM_加暘環保科技有限公司,H51A9221_220099:其他未列名塑膠製品_RP_加暘環保科技有限公司,475,493,H51A9221_220099:其他未列名塑膠製品_RP_加暘環保科技有限公司
3,K8200222_R-0201_RM_連泰紙業股份有限公司,K8200222_183799:其他塑膠粒_RP_連泰紙業股份有限公司,486,497,K8200222_183799:其他塑膠粒_RP_連泰紙業股份有限公司
4,H47A6728_R-0201_RM_柯富伊企業有限公司,H47A6728_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司,481,495,H47A6728_220099:其他未列名塑膠製品_RP_柯富伊企業有限公司
...,...,...,...,...,...
643,A3995069_R-0201_C_達和廢棄物清除股份有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠,433,492,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠
644,A45A1094_R-0201_C_昌久金屬科技有限公司,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠,364,492,H45C2918_R-0201_RM_寶潔廢棄物清除股份有限公司楊梅廠
645,B0200217_R-0201_C_佳霖環保工程有限公司,K8200222_R-0201_RM_連泰紙業股份有限公司,356,486,K8200222_R-0201_RM_連泰紙業股份有限公司
646,B0413983_R-0201_C_世和環保有限公司,H45B6696_R-0201_RM_永茂環科股份有限公司,325,477,H45B6696_R-0201_RM_永茂環科股份有限公司


In [26]:
df_file_2_edge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 647
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   unique_key1  648 non-null    object
 1   unique_key2  648 non-null    object
 2   node_idx_x   648 non-null    int64 
 3   node_idx_y   648 non-null    int64 
 4   unique_key   648 non-null    object
dtypes: int64(2), object(3)
memory usage: 30.4+ KB


In [27]:
df_file_2_edge=df_file_2_edge[['node_idx_x', 'node_idx_y']]
df_file_2_edge

,node_idx_x,node_idx_y
0,477,494
1,482,496
2,475,493
3,486,497
4,481,495
...,...,...
643,433,492
644,364,492
645,356,486
646,325,477


In [28]:
edges=df_file_2_edge.values
edges

array([[477, 494],
       [482, 496],
       [475, 493],
       ...,
       [356, 486],
       [325, 477],
       [317, 477]])

In [29]:
edges.shape

(648, 2)

### Create key 'weights'

In [30]:
!ls

file_1_mapping_new.csv          node-label.csv
file_2_edge_new.csv             nodeidx2uniquekey.csv
file_3_feature.csv              output.json
input_files_preprocessing.ipynb


In [31]:
df_file_2_weight=pd.read_csv('file_2_edge_new.csv')
df_file_2_weight

,fac_no1,code_no1,fac_type1,fac_name1,fac_no2,code_no2,fac_type2,fac_name2,9901,9902,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,H45B6696,R-0201,RM,永茂環科股份有限公司,H45B6696,000099:其他,RP,永茂環科股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H47A9742,R-0201,RM,柯富伊企業有限公司(二廠),H47A9742,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司(二廠),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H51A9221,R-0201,RM,加暘環保科技有限公司,H51A9221,220099:其他未列名塑膠製品,RP,加暘環保科技有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K8200222,R-0201,RM,連泰紙業股份有限公司,K8200222,183799:其他塑膠粒,RP,連泰紙業股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H47A6728,R-0201,RM,柯富伊企業有限公司,H47A6728,220099:其他未列名塑膠製品,RP,柯富伊企業有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,A3995069,R-0201,C,達和廢棄物清除股份有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,A45A1094,R-0201,C,昌久金屬科技有限公司,H45C2918,R-0201,RM,寶潔廢棄物清除股份有限公司楊梅廠,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,B0200217,R-0201,C,佳霖環保工程有限公司,K8200222,R-0201,RM,連泰紙業股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,B0413983,R-0201,C,世和環保有限公司,H45B6696,R-0201,RM,永茂環科股份有限公司,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_file_2_weight=df_file_2_weight.drop(['fac_no1', 'code_no1', 'fac_type1', 'fac_name1', 'fac_no2', 'code_no2', 'fac_type2', 'fac_name2'], axis=1)
df_file_2_weight

,9901,9902,9903,9904,9905,9906,9907,9908,9909,9910,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# replace NaN values with 0
df_file_2_weight=df_file_2_weight.fillna(0)
df_file_2_weight

,9901,9902,9903,9904,9905,9906,9907,9908,9909,9910,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


df_file_2_weight.info(verbose=True, show_counts=True)

In [34]:
weights=np.transpose(df_file_2_weight.values)
weights

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
weights.shape

(168, 648)

In [36]:
len(weights[0])

648

### Create key 'y'

In [37]:
!ls

file_1_mapping_new.csv          node-label.csv
file_2_edge_new.csv             nodeidx2uniquekey.csv
file_3_feature.csv              output.json
input_files_preprocessing.ipynb


In [38]:
df_node_label=pd.read_csv('node-label.csv', header=None)
df_node_label

,0
0,0
1,0
2,0
3,0
4,0
...,...
91723,0
91724,0
91725,0
91726,0


In [39]:
label=df_node_label.values
label

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [40]:
label=label.flatten()
label

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
len(label)

91728

In [42]:
y=[]

for i in range(168): # 168 time periods
    gt=[]
    for j in range(546): # 546 nodes
        gt.append(label[i+j*168])
    y.append(gt)

In [43]:
y=np.array(y)
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
y.shape

(168, 546)

### Create key 'X'

In [45]:
!ls

file_1_mapping_new.csv          node-label.csv
file_2_edge_new.csv             nodeidx2uniquekey.csv
file_3_feature.csv              output.json
input_files_preprocessing.ipynb


In [46]:
df_mapping=pd.read_csv('file_1_mapping_new.csv')
df_mapping

,fac_no,code_no,fac_type,fac_name
0,J59A7436,R-0201,A1,力成科技股份有限公司3C廠
1,J59A0579,R-0201,A1,力成科技股份有限公司大同廠
2,J59B1009,R-0201,A1,力成科技股份有限公司文化一廠
3,O17K9082,R-0201,A1,力成科技股份有限公司竹科分公司
4,J5909547,R-0201,A1,力成科技股份有限公司湖口廠
...,...,...,...,...
541,89523396,220099:其他未列名塑膠製品,S,鴻升包裝器材事業有限公司
542,59132956,183799:其他塑膠粒,S,豐隆塑膠加工廠
543,NaN,183799:其他塑膠粒,S,TIANJIN JIUFU(天津九福國際物流有限公司)
544,NaN,183799:其他塑膠粒,S,Xiamen Luhai (廈門陸海環保股份有限公司)


In [47]:
# replace NaN values with an empty string
df_mapping=df_mapping.fillna('')
df_mapping

,fac_no,code_no,fac_type,fac_name
0,J59A7436,R-0201,A1,力成科技股份有限公司3C廠
1,J59A0579,R-0201,A1,力成科技股份有限公司大同廠
2,J59B1009,R-0201,A1,力成科技股份有限公司文化一廠
3,O17K9082,R-0201,A1,力成科技股份有限公司竹科分公司
4,J5909547,R-0201,A1,力成科技股份有限公司湖口廠
...,...,...,...,...
541,89523396,220099:其他未列名塑膠製品,S,鴻升包裝器材事業有限公司
542,59132956,183799:其他塑膠粒,S,豐隆塑膠加工廠
543,,183799:其他塑膠粒,S,TIANJIN JIUFU(天津九福國際物流有限公司)
544,,183799:其他塑膠粒,S,Xiamen Luhai (廈門陸海環保股份有限公司)


In [48]:
df_file_3_feature=pd.read_csv('file_3_feature.csv')
df_file_3_feature

,fac_no,code_no,fac_type,fac_name,9901,9902,9903,9904,9905,9906,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,10180163,183799:其他塑膠粒,S,鈺格企業社,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
1,10651868,220099:其他未列名塑膠製品,S,尚明塑膠企業社,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
2,10950059,220099:其他未列名塑膠製品,S,金鈺承企業社,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
3,12807682,220099:其他未列名塑膠製品,S,信昊塑膠實業有限公司,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
4,12927997,220099:其他未列名塑膠製品,S,互愉企業有限公司,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,R97A0010,R-0201,A1,尚穩環保科技股份有限公司台南廠,"0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
542,S2000424,R-0201,A1,台灣寶理塑膠股份有限公司大發廠,"0,0,0,32,20","0,0,0,32,20","0,0,0,32,20","0,0,0,32,20","0,0,0,32,20","0,0,0,32,20",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
543,S2213510,R-0201,A1,新世紀環保服務股份有限公司,"0,0,0,200,100","14.32,0,0,200,100","4.25,0,0,200,100","21.27,0,8,200,100","27.46,0,12,200,100","13.54,0,16,200,100",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
544,T42A4310,R-0201,A1,明揚國際科技股份有限公司,"0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"


In [49]:
# replace NaN values with an empty string
df_file_3_feature=df_file_3_feature.fillna('')
df_file_3_feature

,fac_no,code_no,fac_type,fac_name,9901,9902,9903,9904,9905,9906,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,10180163,183799:其他塑膠粒,S,鈺格企業社,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
1,10651868,220099:其他未列名塑膠製品,S,尚明塑膠企業社,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
2,10950059,220099:其他未列名塑膠製品,S,金鈺承企業社,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
3,12807682,220099:其他未列名塑膠製品,S,信昊塑膠實業有限公司,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
4,12927997,220099:其他未列名塑膠製品,S,互愉企業有限公司,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,R97A0010,R-0201,A1,尚穩環保科技股份有限公司台南廠,"0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92","0,0,0,467.4,373.92",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
542,S2000424,R-0201,A1,台灣寶理塑膠股份有限公司大發廠,"0,0,0,32,20","0,0,0,32,20","0,0,0,32,20","0,0,0,32,20","0,0,0,32,20","0,0,0,32,20",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
543,S2213510,R-0201,A1,新世紀環保服務股份有限公司,"0,0,0,200,100","14.32,0,0,200,100","4.25,0,0,200,100","21.27,0,8,200,100","27.46,0,12,200,100","13.54,0,16,200,100",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
544,T42A4310,R-0201,A1,明揚國際科技股份有限公司,"0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056","0,0,0,31.006,25.0056",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"


In [50]:
# align to the order of file_1_mapping
# df_file_1_mapping LEFT JOIN df_file_3_feature
df_file_3_feature=pd.merge(df_mapping, df_file_3_feature, how='left', on=['fac_no', 'code_no', 'fac_type', 'fac_name'])
df_file_3_feature

,fac_no,code_no,fac_type,fac_name,9901,9902,9903,9904,9905,9906,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,J59A7436,R-0201,A1,力成科技股份有限公司3C廠,"0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
1,J59A0579,R-0201,A1,力成科技股份有限公司大同廠,"0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
2,J59B1009,R-0201,A1,力成科技股份有限公司文化一廠,"0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
3,O17K9082,R-0201,A1,力成科技股份有限公司竹科分公司,"0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
4,J5909547,R-0201,A1,力成科技股份有限公司湖口廠,"0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,89523396,220099:其他未列名塑膠製品,S,鴻升包裝器材事業有限公司,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
542,59132956,183799:其他塑膠粒,S,豐隆塑膠加工廠,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
543,,183799:其他塑膠粒,S,TIANJIN JIUFU(天津九福國際物流有限公司),"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
544,,183799:其他塑膠粒,S,Xiamen Luhai (廈門陸海環保股份有限公司),"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"


In [51]:
df_file_3_feature=df_file_3_feature.drop(['fac_no', 'code_no', 'fac_type', 'fac_name'], axis=1)
df_file_3_feature

,9901,9902,9903,9904,9905,9906,9907,9908,9909,9910,...,11203,11204,11205,11206,11207,11208,11209,11210,11211,11212
0,"0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75","0,0,0,1.5,0.75",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
1,"0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1","0,0,0,3,1",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
2,"0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293","0,0,0,6.4,3.293",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
3,"0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2","0,0,0,3,2",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
4,"0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2","0,0,0,4,2",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
542,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
543,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"
544,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0",...,"0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0","0,0,0,0,0"


In [52]:
X=df_file_3_feature.to_numpy()
X

array([['0,0,0,1.5,0.75', '0,0,0,1.5,0.75', '0,0,0,1.5,0.75', ...,
        '0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,3,1', '0,0,0,3,1', '0,0,0,3,1', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,6.4,3.293', '0,0,0,6.4,3.293', '0,0,0,6.4,3.293', ...,
        '0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0'],
       ...,
       ['0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0']], dtype=object)

In [53]:
X=np.transpose(X)
X

array([['0,0,0,1.5,0.75', '0,0,0,3,1', '0,0,0,6.4,3.293', ...,
        '0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,1.5,0.75', '0,0,0,3,1', '0,0,0,6.4,3.293', ...,
        '0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,1.5,0.75', '0,0,0,3,1', '0,0,0,6.4,3.293', ...,
        '0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0'],
       ...,
       ['0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0'],
       ['0,0,0,0,0', '0,0,0,0,0', '0,0,0,0,0', ..., '0,0,0,0,0',
        '0,0,0,0,0', '0,0,0,0,0']], dtype=object)

In [54]:
X.shape

(168, 546)

In [55]:
for i in range(168):
    for j in range(546):
        X[i][j]=X[i][j].split(',')
        
X

array([[list(['0', '0', '0', '1.5', '0.75']),
        list(['0', '0', '0', '3', '1']),
        list(['0', '0', '0', '6.4', '3.293']), ...,
        list(['0', '0', '0', '0', '0']), list(['0', '0', '0', '0', '0']),
        list(['0', '0', '0', '0', '0'])],
       [list(['0', '0', '0', '1.5', '0.75']),
        list(['0', '0', '0', '3', '1']),
        list(['0', '0', '0', '6.4', '3.293']), ...,
        list(['0', '0', '0', '0', '0']), list(['0', '0', '0', '0', '0']),
        list(['0', '0', '0', '0', '0'])],
       [list(['0', '0', '0', '1.5', '0.75']),
        list(['0', '0', '0', '3', '1']),
        list(['0', '0', '0', '6.4', '3.293']), ...,
        list(['0', '0', '0', '0', '0']), list(['0', '0', '0', '0', '0']),
        list(['0', '0', '0', '0', '0'])],
       ...,
       [list(['0', '0', '0', '0', '0']), list(['0', '0', '0', '0', '0']),
        list(['0', '0', '0', '0', '0']), ...,
        list(['0', '0', '0', '0', '0']), list(['0', '0', '0', '0', '0']),
        list(['0', '0', '0', 

In [56]:
for i in range(168):
    for j in range(546):
        X[i][j]=['0' if e=='' else e for e in X[i][j]]
        X[i][j]=[float(e) for e in X[i][j]]
        
X

array([[list([0.0, 0.0, 0.0, 1.5, 0.75]),
        list([0.0, 0.0, 0.0, 3.0, 1.0]),
        list([0.0, 0.0, 0.0, 6.4, 3.293]), ...,
        list([0.0, 0.0, 0.0, 0.0, 0.0]), list([0.0, 0.0, 0.0, 0.0, 0.0]),
        list([0.0, 0.0, 0.0, 0.0, 0.0])],
       [list([0.0, 0.0, 0.0, 1.5, 0.75]),
        list([0.0, 0.0, 0.0, 3.0, 1.0]),
        list([0.0, 0.0, 0.0, 6.4, 3.293]), ...,
        list([0.0, 0.0, 0.0, 0.0, 0.0]), list([0.0, 0.0, 0.0, 0.0, 0.0]),
        list([0.0, 0.0, 0.0, 0.0, 0.0])],
       [list([0.0, 0.0, 0.0, 1.5, 0.75]),
        list([0.0, 0.0, 0.0, 3.0, 1.0]),
        list([0.0, 0.0, 0.0, 6.4, 3.293]), ...,
        list([0.0, 0.0, 0.0, 0.0, 0.0]), list([0.0, 0.0, 0.0, 0.0, 0.0]),
        list([0.0, 0.0, 0.0, 0.0, 0.0])],
       ...,
       [list([0.0, 0.0, 0.0, 0.0, 0.0]), list([0.0, 0.0, 0.0, 0.0, 0.0]),
        list([0.0, 0.0, 0.0, 0.0, 0.0]), ...,
        list([0.0, 0.0, 0.0, 0.0, 0.0]), list([0.0, 0.0, 0.0, 0.0, 0.0]),
        list([0.0, 0.0, 0.0, 0.0, 0.0])],
       [lis

In [57]:
for i in range(168):
    for j in range(546):
        X[i][j]=np.array(X[i][j])
        
X

array([[array([0.  , 0.  , 0.  , 1.5 , 0.75]),
        array([0., 0., 0., 3., 1.]),
        array([0.   , 0.   , 0.   , 6.4  , 3.293]), ...,
        array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]),
        array([0., 0., 0., 0., 0.])],
       [array([0.  , 0.  , 0.  , 1.5 , 0.75]),
        array([0., 0., 0., 3., 1.]),
        array([0.   , 0.   , 0.   , 6.4  , 3.293]), ...,
        array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]),
        array([0., 0., 0., 0., 0.])],
       [array([0.  , 0.  , 0.  , 1.5 , 0.75]),
        array([0., 0., 0., 3., 1.]),
        array([0.   , 0.   , 0.   , 6.4  , 3.293]), ...,
        array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]),
        array([0., 0., 0., 0., 0.])],
       ...,
       [array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.]),
        array([0., 0., 0., 0., 0.]), ..., array([0., 0., 0., 0., 0.]),
        array([0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.])],
       [array([0., 0., 0., 0., 0.]), array([0., 0

In [58]:
X.shape

(168, 546)

In [85]:
# set missing value
X[42][543]=np.array([0.0, 0.0, 0.0, 0.0, 0.0])

In [88]:
X[42][543]

array([0., 0., 0., 0., 0.])

In [89]:
ans=[]

for i in range(168):
    ans.append(np.stack(tuple(X[i][j] for j in range(546)), axis=0))

In [90]:
ans

[array([[0.   , 0.   , 0.   , 1.5  , 0.75 ],
        [0.   , 0.   , 0.   , 3.   , 1.   ],
        [0.   , 0.   , 0.   , 6.4  , 3.293],
        ...,
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ]]),
 array([[0.   , 0.   , 0.   , 1.5  , 0.75 ],
        [0.   , 0.   , 0.   , 3.   , 1.   ],
        [0.   , 0.   , 0.   , 6.4  , 3.293],
        ...,
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ]]),
 array([[0.   , 0.   , 0.   , 1.5  , 0.75 ],
        [0.   , 0.   , 0.   , 3.   , 1.   ],
        [0.   , 0.   , 0.   , 6.4  , 3.293],
        ...,
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ]]),
 array([[0.   , 0.   , 0.   , 1.5  , 0.75 ],
        [0.   , 0.   , 0.   , 3.   , 1.   ],
        [0.   , 0.   , 0.   , 6.4  , 3.293],
        ..

In [91]:
ans=np.array(ans)

In [92]:
ans.shape

(168, 546, 5)

In [93]:
ans[0]

array([[0.   , 0.   , 0.   , 1.5  , 0.75 ],
       [0.   , 0.   , 0.   , 3.   , 1.   ],
       [0.   , 0.   , 0.   , 6.4  , 3.293],
       ...,
       [0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ]])

In [77]:
np.stack(tuple(X[0][i] for i in range(546)), axis=0)

array([[0.   , 0.   , 0.   , 1.5  , 0.75 ],
       [0.   , 0.   , 0.   , 3.   , 1.   ],
       [0.   , 0.   , 0.   , 6.4  , 3.293],
       ...,
       [0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   ]])

## Create dictionary

In [94]:
outer_dict={}

for i in range(168): # 168 time periods
    inner_dict={}
    
    inner_dict['index']=i
    inner_dict['edges']=edges
    inner_dict['weights']=weights[i]
    inner_dict['y']=y[i]
    inner_dict['X']=ans[i]
    
    outer_dict[str(i)]=inner_dict
    
outer_dict

{'0': {'index': 0,
  'edges': array([[477, 494],
         [482, 496],
         [475, 493],
         ...,
         [356, 486],
         [325, 477],
         [317, 477]]),
  'weights': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [95]:
outer_dict['time_periods']=168
outer_dict['node_ids']=node_ids

outer_dict

{'0': {'index': 0,
  'edges': array([[477, 494],
         [482, 496],
         [475, 493],
         ...,
         [356, 486],
         [325, 477],
         [317, 477]]),
  'weights': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [96]:
type(outer_dict)

dict

In [97]:
# convert from Python to JSON
# I found the best solution if you have nested numpy arrays in a dictionary:
import json

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [98]:
with open('output.json', 'w', encoding='utf8') as f:
    json.dump(outer_dict, f, ensure_ascii=False, cls=NumpyEncoder)